#### Development of multitask sequence labeling model for two tasks, where 
 - Task 1 is word-level sequence labeling.
 - Task 2 is sentence-level labeling. Prepare a model in Keras that performs both word and sentence-level  labeling simultaneously.

### Creating a Datasets

In [103]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Toy dataset for word-level NER task
sentences = [
    "John lives in New York.",
    "Apple is a company based in California.",
    "I love my dog.",
    "The Eiffel Tower is in Paris."
]

ner_labels = [
    ["B-PER", "O", "O", "B-LOC", "B-LOC"],
    ["B-ORG", "O", "O", "O", "O", "O", "B-LOC"],
    ["O", "O", "O", "O"],
    ["O", "B-LOC", "O", "O", "O", "B-LOC"]
]

# Sentiment labels (0: Negative, 1: Neutral, 2: Positive)
sentiment_labels = [2, 1, 2, 2]

### Process Data

In [104]:
# Create vocabulary and label sets
vocab = set(word for sentence in sentences for word in sentence.split())
ner_tags = set(tag for tags in ner_labels for tag in tags)
num_classes_ner = len(ner_tags)
num_classes_sentiment = 3  # Negative, Neutral, Positive

# Create word and label dictionaries
word2idx = {word: idx + 1 for idx, word in enumerate(vocab)}
ner2idx = {tag: idx for idx, tag in enumerate(ner_tags)}

# Convert sentences to numerical sequences and labels to numerical sequences
X_word = [[word2idx[word] for word in sentence.split()] for sentence in sentences]
y_ner = [[ner2idx[tag] for tag in tags] for tags in ner_labels]

# Pad sequences
max_sequence_length = max(len(seq) for seq in X_word)
X_word = pad_sequences(X_word, maxlen=max_sequence_length, padding='post')
y_ner = pad_sequences(y_ner, maxlen=max_sequence_length, padding='post')

# Convert NER labels to one-hot encoded format
y_ner = np.array([to_categorical(seq, num_classes=num_classes_ner) for seq in y_ner])
y_sentiment = to_categorical(sentiment_labels, num_classes=num_classes_sentiment)

### Building Model

In [105]:
from keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, GlobalMaxPooling1D
from keras.models import Model

# Input layer
word_input = Input(shape=(max_sequence_length,))

# Shared embedding layer
embedding_dim = 50
embedding_layer = Embedding(input_dim=len(vocab) + 1, output_dim=embedding_dim, input_length=max_sequence_length)
word_embed = embedding_layer(word_input)

# Shared LSTM layer
lstm_units = 64
lstm_layer = LSTM(lstm_units, return_sequences=True)
word_lstm = lstm_layer(word_embed)

# Word-level NER output
ner_output = TimeDistributed(Dense(num_classes_ner, activation='softmax'))(word_lstm)

# Sentence-level Sentiment output
sentence_lstm = LSTM(lstm_units)(word_lstm)
sentiment_output = Dense(num_classes_sentiment, activation='softmax')(sentence_lstm)

# Create the multitask model
model = Model(inputs=word_input, outputs=[ner_output, sentiment_output])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Model summary:")
model.summary()


Model summary:
Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 7)]          0           []                               
                                                                                                  
 embedding_15 (Embedding)       (None, 7, 50)        1000        ['input_21[0][0]']               
                                                                                                  
 lstm_23 (LSTM)                 (None, 7, 64)        29440       ['embedding_15[0][0]']           
                                                                                                  
 lstm_24 (LSTM)                 (None, 64)           33024       ['lstm_23[0][0]']                
                                                                            

In [119]:
# Train the model
model.fit(X_word, [y_ner, y_sentiment], epochs=90, batch_size=2, verbose=1)

Epoch 1/90
2/2 [==============================] - 0s 40ms/step - loss: 0.1077 - time_distributed_12_loss: 0.1076 - dense_23_loss: 3.7073e-05 - time_distributed_12_accuracy: 0.9643 - dense_23_accuracy: 1.0000
Epoch 2/90
2/2 [==============================] - 0s 24ms/step - loss: 0.1056 - time_distributed_12_loss: 0.1056 - dense_23_loss: 3.7043e-05 - time_distributed_12_accuracy: 0.9643 - dense_23_accuracy: 1.0000
Epoch 3/90
2/2 [==============================] - 0s 24ms/step - loss: 0.1039 - time_distributed_12_loss: 0.1039 - dense_23_loss: 3.6804e-05 - time_distributed_12_accuracy: 0.9643 - dense_23_accuracy: 1.0000
Epoch 4/90
2/2 [==============================] - 0s 32ms/step - loss: 0.1020 - time_distributed_12_loss: 0.1020 - dense_23_loss: 3.6387e-05 - time_distributed_12_accuracy: 0.9643 - dense_23_accuracy: 1.0000
Epoch 5/90
2/2 [==============================] - 0s 32ms/step - loss: 0.1004 - time_distributed_12_loss: 0.1004 - dense_23_loss: 3.6059e-05 - time_distributed_12_accur

2/2 [==============================] - 0s 32ms/step - loss: 0.0537 - time_distributed_12_loss: 0.0537 - dense_23_loss: 2.7000e-05 - time_distributed_12_accuracy: 1.0000 - dense_23_accuracy: 1.0000
Epoch 41/90
2/2 [==============================] - 0s 24ms/step - loss: 0.0530 - time_distributed_12_loss: 0.0530 - dense_23_loss: 2.6762e-05 - time_distributed_12_accuracy: 1.0000 - dense_23_accuracy: 1.0000
Epoch 42/90
2/2 [==============================] - 0s 24ms/step - loss: 0.0523 - time_distributed_12_loss: 0.0523 - dense_23_loss: 2.6523e-05 - time_distributed_12_accuracy: 1.0000 - dense_23_accuracy: 1.0000
Epoch 43/90
2/2 [==============================] - 0s 32ms/step - loss: 0.0515 - time_distributed_12_loss: 0.0514 - dense_23_loss: 2.6225e-05 - time_distributed_12_accuracy: 1.0000 - dense_23_accuracy: 1.0000
Epoch 44/90
2/2 [==============================] - 0s 49ms/step - loss: 0.0508 - time_distributed_12_loss: 0.0507 - dense_23_loss: 2.6106e-05 - time_distributed_12_accuracy: 1.

2/2 [==============================] - 0s 24ms/step - loss: 0.0312 - time_distributed_12_loss: 0.0311 - dense_23_loss: 2.0265e-05 - time_distributed_12_accuracy: 1.0000 - dense_23_accuracy: 1.0000
Epoch 80/90
2/2 [==============================] - 0s 16ms/step - loss: 0.0307 - time_distributed_12_loss: 0.0307 - dense_23_loss: 2.0146e-05 - time_distributed_12_accuracy: 1.0000 - dense_23_accuracy: 1.0000
Epoch 81/90
2/2 [==============================] - 0s 24ms/step - loss: 0.0302 - time_distributed_12_loss: 0.0302 - dense_23_loss: 2.0057e-05 - time_distributed_12_accuracy: 1.0000 - dense_23_accuracy: 1.0000
Epoch 82/90
2/2 [==============================] - 0s 32ms/step - loss: 0.0297 - time_distributed_12_loss: 0.0297 - dense_23_loss: 1.9967e-05 - time_distributed_12_accuracy: 1.0000 - dense_23_accuracy: 1.0000
Epoch 83/90
2/2 [==============================] - 0s 24ms/step - loss: 0.0292 - time_distributed_12_loss: 0.0292 - dense_23_loss: 1.9788e-05 - time_distributed_12_accuracy: 1.

### Make Prediction

In [120]:
# Make predictions
ner_predictions, sentiment_predictions = model.predict(X_word)

# Print sample predictions
sample_idx = 1
sample_sentence = sentences[sample_idx]
sample_ner_pred = np.argmax(ner_predictions[sample_idx], axis=-1)
sample_sentiment_pred = np.argmax(sentiment_predictions[sample_idx])

print("Sample Sentence:", sample_sentence)
print("NER Predictions:", [list(ner2idx.keys())[idx] for idx in sample_ner_pred])
print("Sentiment Prediction:", sample_sentiment_pred)


1/1 [==============================] - 0s 65ms/step
Sample Sentence: Apple is a company based in California.
NER Predictions: ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-LOC']
Sentiment Prediction: 1


In [123]:
# Make predictions
ner_predictions, sentiment_predictions = model.predict(X_word)

# Print sample predictions
sample_idx = 2
sample_sentence = sentences[sample_idx]
sample_ner_pred = np.argmax(ner_predictions[sample_idx], axis=-1)
sample_sentiment_pred = np.argmax(sentiment_predictions[sample_idx])

print("Sample Sentence:", sample_sentence)
print("NER Predictions:", [list(ner2idx.keys())[idx] for idx in sample_ner_pred])
print("Sentiment Prediction:", sample_sentiment_pred)


1/1 [==============================] - 0s 72ms/step
Sample Sentence: I love my dog.
NER Predictions: ['O', 'O', 'O', 'O', 'B-LOC', 'B-LOC', 'B-LOC']
Sentiment Prediction: 2
